In [12]:
using LowRankApprox
using Plots
using LinearAlgebra
using Jacobi
using Test
gr()
Plots.GRBackend()

Plots.GRBackend()

In [13]:
function TrilinearMap(coord_E, xhat, yhat, zhat)

    
    m = length(xhat)
    P1 = @. 0.5*(1-xhat-yhat)*(1-zhat)
    P2 = @. 0.5*xhat*(1-zhat)
    P3 = @. 0.5*yhat*(1-zhat)
    P4 = @. 0.5*(1-xhat-yhat)*(1+zhat)
    P5 = @. 0.5*xhat*(1+zhat)
    P6 = @. 0.5*yhat*(1+zhat)
    P = [P1 P2 P3 P4 P5 P6]
    P_E = P * coord_E
    # x=F_E(xhat)
    X = P_E
    x = X[:, 1]
    y = X[:, 2]
    z = X[:, 3]
    # gradient of P, [dP/dxhat; dP/dyhat; dP/dzhat]
    GradP = @. [-0.5*(1-zhat) 0.5*(1-zhat) 0 -0.5*(1+zhat) 0.5*(1+zhat) 0;
                -0.5*(1-zhat) 0 0.5*(1-zhat) -0.5*(1+zhat) 0 0.5*(1+zhat);
                -0.5*(1-xhat-yhat) -0.5*xhat -0.5*yhat 0.5*(1-xhat-yhat) 0.5*xhat 0.5*yhat]

    # JT = [[dx/dxhat, dy/dxhat, dz/dxhat],
    #       [dx/dyhat, dy/dyhat, dz/dyhat],
    #       [dx/dzhat, dy/dzhat, dz/dzhat]] (3m x 3)

    JT = GradP * coord_E
    dxdxhat = JT[1:m,1]
    dydxhat = JT[1:m,2]
    dzdxhat = JT[1:m,3]
    dxdyhat = JT[m+1:2*m,1]
    dydyhat = JT[m+1:2*m,2]
    dzdyhat = JT[m+1:2*m,3]
    dxdzhat = JT[2*m+1:3*m,1]
    dydzhat = JT[2*m+1:3*m,2]
    dzdzhat = JT[2*m+1:3*m,3]
    J_E = @. (dxdxhat*dydyhat*dzdzhat + dxdyhat*dydzhat*dzdzhat + dxdzhat*dydxhat*dzdyhat)-(dxdxhat*dydzhat*dzdyhat + dxdyhat*dydxhat*dzdzhat + dxdzhat*dydyhat*dzdxhat)
    
    DF_E = [dxdxhat dxdyhat dxdzhat;dydxhat dydyhat dydzhat;dzdxhat dzdyhat dzdzhat]
    
    return x, y, z, DF_E, J_E
end

TrilinearMap (generic function with 1 method)

In [14]:
coord_E = [0. 0. 0.;1. 0. 0.;.5 .5 0.;
               0. 0. 1.;1. 0. 1.;.5 .5 1.]
xhat = [0]
yhat = [1]
zhat = [1]
x, y, z, DF_E, J_E = TrilinearMap(coord_E,xhat,yhat,zhat)

([0.5], [0.5], [1.0], [1.0 0.5 0.0; 0.0 0.5 0.0; 0.0 0.0 0.5], [0.25])

In [15]:
function PrimeBasis(coord_E, xhat, yhat, zhat)

    m = length(xhat)
    # supplements from Wheeler, Xue, & Yotov, 2011
    shat1 = [(2*yhat .* zhat) (zeros(m)) (zeros(m))]
    shat2 = [(zeros(m)) (2*xhat .* zhat) (zeros(m))]
    shat3 = [(zeros(m)) (2*yhat .* zhat) (-zhat.^2)]  
    shat4 = [(2*xhat .* zhat) (zeros(m)) (-zhat.^2)]
    shat5 = [(xhat.^2) (xhat .* yhat) (-3*xhat .* zhat)]
    shat6 = [(yhat .* xhat) (yhat.^2) (-3*yhat .* zhat)]

    # (x,y,z) are in E
    x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
    
    # we have 18 basis in 3D space
    P1 = zeros(3,m,18)
    P1[1,:,1] = ones(m)
    P1[1,:,2] = x
    P1[1,:,3] = y
    P1[1,:,4] = z
    P1[2,:,5] = ones(m)
    P1[2,:,6] = x
    P1[2,:,7] = y
    P1[2,:,8] = z
    P1[3,:,9] = ones(m)
    P1[3,:,10] = x
    P1[3,:,11] = y
    P1[3,:,12] = z
    # supplement
    P1[1,:,13] = (DF_E[1:m,1] .* shat1[:,1] + DF_E[1:m,2] .* shat1[:,2] + DF_E[1:m,3] .* shat1[:,3]) ./ J_E
    P1[2,:,13] = (DF_E[m+1:2*m,1] .* shat1[:,1] + DF_E[m+1:2*m,2] .* shat1[:,2] + DF_E[m+1:2*m,3] .* shat1[:,3]) ./ J_E
    P1[3,:,13] = (DF_E[2*m+1:3*m,1] .* shat1[:,1] + DF_E[2*m+1:3*m,2] .* shat1[:,2] + DF_E[2*m+1:3*m,3] .* shat1[:,3]) ./ J_E

    P1[1,:,14] = (DF_E[1:m,1] .* shat2[:,1] + DF_E[1:m,2] .* shat2[:,2] + DF_E[1:m,3] .* shat2[:,3]) ./ J_E
    P1[2,:,14] = (DF_E[m+1:2*m,1] .* shat2[:,1] + DF_E[m+1:2*m,2] .* shat2[:,2] + DF_E[m+1:2*m,3] .* shat2[:,3]) ./ J_E
    P1[3,:,14] = (DF_E[2*m+1:3*m,1] .* shat2[:,1] + DF_E[2*m+1:3*m,2] .* shat2[:,2] + DF_E[2*m+1:3*m,3] .* shat2[:,3]) ./ J_E

    P1[1,:,15] = (DF_E[1:m,1] .* shat3[:,1] + DF_E[1:m,2] .* shat3[:,2] + DF_E[1:m,3] .* shat3[:,3]) ./ J_E
    P1[2,:,15] = (DF_E[m+1:2*m,1] .* shat3[:,1] + DF_E[m+1:2*m,2] .* shat3[:,2] + DF_E[m+1:2*m,3] .* shat3[:,3]) ./ J_E
    P1[3,:,15] = (DF_E[2*m+1:3*m,1] .* shat3[:,1] + DF_E[2*m+1:3*m,2] .* shat3[:,2] + DF_E[2*m+1:3*m,3] .* shat3[:,3]) ./ J_E

    P1[1,:,16] = (DF_E[1:m,1] .* shat4[:,1] + DF_E[1:m,2] .* shat4[:,2] + DF_E[1:m,3] .* shat4[:,3]) ./ J_E
    P1[2,:,16] = (DF_E[m+1:2*m,1] .* shat4[:,1] + DF_E[m+1:2*m,2] .* shat4[:,2] + DF_E[m+1:2*m,3] .* shat4[:,3]) ./ J_E
    P1[3,:,16] = (DF_E[2*m+1:3*m,1] .* shat4[:,1] + DF_E[2*m+1:3*m,2] .* shat4[:,2] + DF_E[2*m+1:3*m,3] .* shat4[:,3]) ./ J_E

    P1[1,:,17] = (DF_E[1:m,1] .* shat5[:,1] + DF_E[1:m,2] .* shat5[:,2] + DF_E[1:m,3] .* shat5[:,3]) ./ J_E
    P1[2,:,17] = (DF_E[m+1:2*m,1] .* shat5[:,1] + DF_E[m+1:2*m,2] .* shat5[:,2] + DF_E[m+1:2*m,3] .* shat5[:,3]) ./ J_E
    P1[3,:,17] = (DF_E[2*m+1:3*m,1] .* shat5[:,1] + DF_E[2*m+1:3*m,2] .* shat5[:,2] + DF_E[2*m+1:3*m,3] .* shat5[:,3]) ./ J_E

    P1[1,:,18] = (DF_E[1:m,1] .* shat6[:,1] + DF_E[1:m,2] .* shat6[:,2] + DF_E[1:m,3] .* shat6[:,3]) ./ J_E
    P1[2,:,18] = (DF_E[m+1:2*m,1] .* shat6[:,1] + DF_E[m+1:2*m,2] .* shat6[:,2] + DF_E[m+1:2*m,3] .* shat6[:,3]) ./ J_E
    P1[3,:,18] = (DF_E[2*m+1:3*m,1] .* shat6[:,1] + DF_E[2*m+1:3*m,2] .* shat6[:,2] + DF_E[2*m+1:3*m,3] .* shat6[:,3]) ./ J_E

    P = zeros(3*m,18)    
    P[1:m,:] = P1[1,:,:]
    P[m+1:2*m,:] = P1[2,:,:]
    P[2*m+1:3*m,:] = P1[3,:,:]
    
    return P
end

PrimeBasis (generic function with 1 method)

In [16]:
function GetNormal(coord_E, xhat, yhat, zhat, edge)

    m = length(xhat)
    if edge == "left"

        n = [0. ; -1. ; 0.] 

    elseif edge == "diagonal"

        n = [sqrt(2)/2 ; sqrt(2)/2; 0.]

    elseif edge == "bottom"
        
        n = [0. ; 0. ; -1.]

    elseif edge == "top"
        
        n = [0. ; 0. ; 1.]
        
    elseif edge == "back"
        
        n = [-1. ; 0. ; 0.]

    else
        println("Error! Enter correct name for edge")
    
    end

    return n
end

GetNormal (generic function with 1 method)

In [17]:
function VondermondeMat(coord_E)
    """
    Input:
    ------
    coord_E: is the coordinate of vertices of element.
    Note
    3---4
    |   |
    1---2
    Output:
    ------
    VM: the 8x8 vondermonde matrix
    """
    nl = GetNormal(coord_E, [0.], [-1.],[0.],"left")
    nd = GetNormal(coord_E, [0.], [1.],[0.],"diagonal")
    nbt = GetNormal(coord_E, [0.], [0.],[-1.],"bottom")
    nt = GetNormal(coord_E, [0.], [0.],[1.],"top")
    nbk = GetNormal(coord_E, [-1.], [0.],[0.],"back")
    normals = [nbk nl nbt nd nl nbt nbk nd nbt nbk nl nt nd nl nt nbk nd nt]
    nd1 = [0.;0.;-1.]
    nd2 = [1.;0.;-1.]
    nd3 = [0.;1.;-1.]
    nd4 = [0.;0.;1.]
    nd5 = [1.;0.;1.]
    nd6 = [0.;1.;1.]
    nodes = [nd1 nd1 nd1 nd2 nd2 nd2 nd3 nd3 nd3 nd4 nd4 nd4 nd5 nd5 nd5 nd6 nd6 nd6]
    # vondermonde matrix, V_ij = phi_j(x_i).n_i
    VM = zeros(18,18)

    for i=1:18
        for j=1:18
            P = PrimeBasis(coord_E, nodes[1,i], nodes[2,i], nodes[3,i])
            VM[i,j] = dot(P[:,j],normals[:,i])
        end
    end

    return VM
end

VondermondeMat (generic function with 1 method)

In [18]:
function GetPrismNodalBasis(coord_E, xhat, yhat, zhat)
   
    VM = VondermondeMat(coord_E)
    P = PrimeBasis(coord_E, xhat, yhat, zhat)
    invVM = inv(VM)
    Nhat = P * invVM

    return Nhat
end

GetPrismNodalBasis (generic function with 1 method)

In [19]:
@testset "TestNodalBasisUniform" begin

    coord_E = [0. 0. -2.;2. 0. -2.;0. 2. -2.;
               0. 0. 2.;2. 0. 2.;0. 2. 2.]
    nl = GetNormal(coord_E, [0.], [-1.],[0.],"left")
    nd = GetNormal(coord_E, [0.], [1.],[0.],"diagonal")
    nbt = GetNormal(coord_E, [0.], [0.],[-1.],"bottom")
    nt = GetNormal(coord_E, [0.], [0.],[1.],"top")
    nbk = GetNormal(coord_E, [-1.], [0.],[0.],"back")
    
    err = 1e-14
    #============= Node 1==============#
    # check node 1
    Nhat = GetPrismNodalBasis(coord_E,[0.], [0.], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 1.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    
    #============= Node 2==============#
    # check node 1
    Nhat = GetPrismNodalBasis(coord_E,[1.], [0.], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nd), 1.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 1.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    #============= Node 3==============#
    # check node 1
    Nhat = GetPrismNodalBasis(coord_E,[0.], [1.], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,8],nd), 1.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 1.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    #============= Node 4==============#
    # check node 1
    Nhat = GetPrismNodalBasis(coord_E,[0.], [0.], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,11],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,12],nt), 1.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    #============= Node 5==============#
    # check node 1
    Nhat = GetPrismNodalBasis(coord_E,[1.], [0.], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nd), 1.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 1.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    #============= Node 6==============#
    # check node 1
    Nhat = GetPrismNodalBasis(coord_E,[0.], [1.], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nd), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,17],nd), 1.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 1.;atol=err)
    
end

Test Summary:         | Pass  Total
TestNodalBasisUniform |  108    108


Test.DefaultTestSet("TestNodalBasisUniform", Any[], 108, false, false)